In [1]:
import numpy as np
import os
import cv2
import keras
import math
import tensorflow as tf
import datetime

In [5]:
tf.__version__

'2.15.0'

In [6]:
keras.__version__

'2.15.0'

In [17]:
class ShopliftingPrediction:
  def __init__(self, model_path, frame_width, frame_height, sequence_length):
    self.frame_width = frame_width
    self.frame_height = frame_height
    self.sequence_length = sequence_length
    self.model_path = model_path
  def load_model(self):
    self.model = tf.keras.models.load_model(self.model_path)
  def generate_message_content(self, probability, label):
    if label == 0:
      if probability <= 75:
        self.message = "There is little chance of theft"
      elif probability <= 85:
        self.message = "High probability of theft"
      else:
        self.message = "Very high probability of theft"
    elif label == 1:
      if probability <= 75:
        self.message = "The movement is confusing, watch"
      elif probability <= 85:
        self.message = "I think it's normal, but it's better to watch"
      else:
        self.message = "Movement is normal"
  def Pre_Process_Video(self, current_frame, previous_frame):
      diff = cv2.absdiff(current_frame, previous_frame)
      diff = cv2.GaussianBlur(diff,(3,3), 0)
      resized_frame = cv2.resize(diff, (self.frame_height, self.frame_width))
      gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY )
      normalized_frame = gray_frame / 255
      return normalized_frame
  def Read_Video(self, filePath):
      self.video_reader = cv2.VideoCapture(filePath)
      self.original_video_width = int(self.video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
      self.original_video_height = int(self.video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
      self.fps = self.video_reader.get(cv2.CAP_PROP_FPS)
  def Single_Frame_Predict(self, frames_queue):
      probabilities = self.model.predict(np.expand_dims(frames_queue, axis = 0))[0]
      predicted_label = np.argmax(probabilities)
      probability = math.floor(max(probabilities[0], probabilities[1])*100)
      return [probability, predicted_label]
  def Predict_Video(self, video_file_path, output_file_path):
      global fps
      global message
      message = 'I will start analysis video now'
      self.Read_Video(video_file_path)
      video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'),
                                    self.fps, (self.original_video_width, self.original_video_height))
      success, frame = self.video_reader.read()
      previous = frame.copy()
      frames_queue = []
      while self.video_reader.isOpened():
          ok, frame = self.video_reader.read()
          if not ok:
              break
          normalized_frame = self.Pre_Process_Video(frame, previous)
          previous = frame.copy()
          frames_queue.append(normalized_frame)
          if len(frames_queue) == self.sequence_length:
              [probability, predicted_label] = self.Single_Frame_Predict(frames_queue)
              self.generate_message_content(probability, predicted_label)
              message = "{}:{}%".format(self.message, probability)
              frames_queue = []
              print(message)
          cv2.rectangle(frame, (0,0), (640, 40), (255, 255, 255), -1)
          cv2.putText(frame, message, (1,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
          video_writer.write(frame)
      self.video_reader.release()
      video_writer.release()

In [18]:
m = ShopliftingPrediction(model_path = "lrcn_160S_90_90Q.h5",
                          frame_width = 90, frame_height = 90, sequence_length = 160)
m.load_model()

In [19]:
input_video_file_path = "test5.mp4"
output_video_file_path = 'output.mp4'
m.Predict_Video(input_video_file_path, output_video_file_path)

1/1 [==============================] - 1s 1s/step
Movement is normal:98%
1/1 [==============================] - 0s 70ms/step
Movement is normal:99%
1/1 [==============================] - 0s 70ms/step
I think it's normal, but it's better to watch:85%
1/1 [==============================] - 0s 63ms/step
The movement is confusing, watch:63%
1/1 [==============================] - 0s 70ms/step
Movement is normal:95%
1/1 [==============================] - 0s 60ms/step
There is little chance of theft:54%
1/1 [==============================] - 0s 70ms/step
Movement is normal:99%
1/1 [==============================] - 0s 70ms/step
Very high probability of theft:98%
1/1 [==============================] - 0s 81ms/step
Very high probability of theft:98%
1/1 [==============================] - 0s 62ms/step
Movement is normal:99%
